In [1]:
# import necessary libraries
import pandas as pd
import numpy as np


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [67]:
# Evaluation of pre-trained models
# Load the dataset
df = pd.read_csv('../results/single_models/master_results.csv')
df = df[(df['testing_model_type'] == 'B') | (df['testing_model_type'] == 'S-B')].drop(columns=['testing_model_augmentation'])
df1 = df[(df['testing_model_idx'] == df['D_idx']) &
         (df['testing_model_structure'] == df['D_generating_model'])]
df2 = df[(df['testing_model_idx'] != df['D_idx']) |
         (df['testing_model_structure'] != df['D_generating_model'])]
group_columns = [col for col in df2.columns if col not in ['testing_model_idx','acc','loss']]
df2_aggregated = df2.groupby(group_columns).agg({'acc': 'mean', 'loss': 'mean'}).reset_index()
group_columns = [col for col in df2_aggregated.columns if col not in ['D_idx','acc','loss']]
df2_aggregated = df2_aggregated.groupby(group_columns).agg({'acc': 'mean', 'loss': 'mean'}).reset_index()
df2_aggregated.rename(columns={'acc': 'avg_acc', 'loss': 'avg_loss'}, inplace=True)
df2_aggregated['testing_model'] = df2_aggregated['testing_model_structure'] + '_' + df2_aggregated['testing_model_type']
df2_aggregated.drop(columns=['testing_model_structure', 'testing_model_type'], inplace=True)

In [ ]:
df2_aggregated['testing_dataset']=df2_aggregated['D_attack_type'] + '_' + df2_aggregated['D_attack_direction']+'_' + df2_aggregated['D_generating_model']
df2_aggregated.drop(columns=['D_attack_type', 'D_attack_direction', 'D_generating_model'], inplace=True)


KeyError: 'D_attack_type'

In [79]:
df2_aggregated.sort_values(by=['testing_dataset'], inplace=True)
df2_aggregated

,avg_acc,avg_loss,testing_model,testing_dataset
0,0.807587,0.872432,ResNet20v1_B,cwl2_targeted_2nd_ResNet56v1
84,0.803367,0.948349,ResNet56v1_B,cwl2_targeted_2nd_ResNet56v1
63,0.894825,0.518341,ResNet20v2_S-B,cwl2_targeted_2nd_ResNet56v1
126,0.768975,1.013869,ResNet56v2_B,cwl2_targeted_2nd_ResNet56v1
42,0.764275,1.008381,ResNet20v2_B,cwl2_targeted_2nd_ResNet56v1
...,...,...,...,...
62,0.790894,1.409884,ResNet20v2_B,pgd_untargeted_vgg19
41,0.848597,1.018563,ResNet20v1_S-B,pgd_untargeted_vgg19
20,0.807472,1.507765,ResNet20v1_B,pgd_untargeted_vgg19
146,0.791331,1.423284,ResNet56v2_B,pgd_untargeted_vgg19


In [97]:
pivot_df = df2_aggregated.drop(columns=['avg_loss']).pivot(index='testing_model', columns='testing_dataset', values = 'avg_acc')
pivot_df_aggregated = pivot_df.groupby(pivot_df.index.str.endswith('S-B')).agg(['mean']).reset_index()

In [104]:
pivot_df_aggregated.rename(index = {0: 'B', 1: 'S-B'}, inplace=True)
pivot_df_aggregated.drop(columns=['index'], inplace=True)

/var/folders/gt/llhq12_57w5fdzbm5g__rpj00000gn/T/ipykernel_18060/1957023840.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pivot_df_aggregated.drop(columns=['index'], inplace=True)


In [ ]:
pivot_df_aggregated.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in pivot_df.columns]


In [116]:
pivot_df_aggregated.transpose().sort_values(by='B').to_csv('../results/single_models/avg_of_B_and_SB.csv')


In [118]:
pivot_df_aggregated

,cwl2_targeted_2nd_ResNet56v1,cwl2_targeted_ll_ResNet56v1,cwl2_untargeted_ResNet56v1,original_original_original,pgd_targeted_2nd_ResNet56v1,pgd_targeted_ll_ResNet20v1,pgd_targeted_ll_ResNet20v2,pgd_targeted_ll_ResNet56v1,pgd_targeted_ll_ResNet56v2,pgd_targeted_ll_vgg11,...,pgd_targeted_ll_vgg16,pgd_targeted_ll_vgg19,pgd_untargeted_ResNet20v1,pgd_untargeted_ResNet20v2,pgd_untargeted_ResNet56v1,pgd_untargeted_ResNet56v2,pgd_untargeted_vgg11,pgd_untargeted_vgg13,pgd_untargeted_vgg16,pgd_untargeted_vgg19
B,0.786051,0.824821,0.513648,0.834513,0.375219,0.510594,0.531371,0.506472,0.526864,0.654498,...,0.522184,0.522727,0.784741,0.770259,0.664969,0.772310,0.785372,0.805976,0.799841,0.800543
S-B,0.894869,0.906602,0.577115,0.909803,0.561091,0.710166,0.725438,0.710215,0.723670,0.795841,...,0.693613,0.692869,0.855486,0.848475,0.768678,0.850339,0.864712,0.861938,0.844272,0.856291


In [51]:
df1[df1['testing_model_structure'] == 'ResNet20v2'].sort_values(by=['testing_model_type','D_attack_direction', 
                 'D_generating_model', 'D_attack_type', 'D_idx',]).head(60)

,testing_model_structure,testing_model_type,testing_model_idx,D_attack_type,D_attack_direction,D_generating_model,D_idx,acc,loss
10452,ResNet20v2,B,0,pgd,targeted_ll,ResNet20v2,0,0.1810,6.695107
10530,ResNet20v2,B,1,pgd,targeted_ll,ResNet20v2,1,0.2124,5.867753
10589,ResNet20v2,B,2,pgd,targeted_ll,ResNet20v2,2,0.1973,6.222771
10679,ResNet20v2,B,3,pgd,targeted_ll,ResNet20v2,3,0.2070,6.413485
10458,ResNet20v2,B,0,pgd,untargeted,ResNet20v2,0,0.7490,6.269812
10522,ResNet20v2,B,1,pgd,untargeted,ResNet20v2,1,0.7351,6.618278
10597,ResNet20v2,B,2,pgd,untargeted,ResNet20v2,2,0.7461,6.426787
10669,ResNet20v2,B,3,pgd,untargeted,ResNet20v2,3,0.7488,6.418821
8872,ResNet20v2,S-B,0,pgd,targeted_ll,ResNet20v2,0,0.5746,2.060593
8950,ResNet20v2,S-B,1,pgd,targeted_ll,ResNet20v2,1,0.5984,1.871910


In [ ]:
df1['testing_dataset']=df1['D_attack_type'] + '_' + df1['D_attack_direction']+'_' + df1['D_generating_model']
df1.drop(columns=['D_attack_type', 'D_attack_direction', 'D_generating_model'], inplace=True)


/var/folders/gt/llhq12_57w5fdzbm5g__rpj00000gn/T/ipykernel_18060/1469924027.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['testing_dataset']=df1['D_attack_type'] + '_' + df1['D_attack_direction']+'_' + df1['D_generating_model']
/var/folders/gt/llhq12_57w5fdzbm5g__rpj00000gn/T/ipykernel_18060/1469924027.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.drop(columns=['D_attack_type', 'D_attack_direction', 'D_generating_model'], inplace=True)


,testing_model_structure,testing_model_type,testing_model_idx,D_idx,acc,loss,testing_dataset
8221,ResNet20v1,S-B,0,0,0.5111,2.609387,pgd_targeted_ll_ResNet20v1
8256,ResNet20v1,S-B,0,0,0.7688,2.357672,pgd_untargeted_ResNet20v1
8296,ResNet20v1,S-B,1,1,0.4822,2.742721,pgd_targeted_ll_ResNet20v1
8315,ResNet20v1,S-B,1,1,0.7871,2.405715,pgd_untargeted_ResNet20v1
8403,ResNet20v1,S-B,2,2,0.4914,2.671431,pgd_targeted_ll_ResNet20v1
...,...,...,...,...,...,...,...
11146,ResNet56v2,B,1,1,0.2025,6.402960,pgd_targeted_ll_ResNet56v2
11220,ResNet56v2,B,2,2,0.2060,6.107372,pgd_targeted_ll_ResNet56v2
11263,ResNet56v2,B,2,2,0.7305,6.821687,pgd_untargeted_ResNet56v2
11307,ResNet56v2,B,3,3,0.7499,6.307493,pgd_untargeted_ResNet56v2


In [ ]:
df1['testing_model'] = df1['testing_model_structure'] + '_' + df1['testing_model_type']+'_' + df1['testing_model_idx'].astype(str)
df1.drop(columns=['testing_model_structure', 'testing_model_type', 'testing_model_idx'], inplace=True)


/var/folders/gt/llhq12_57w5fdzbm5g__rpj00000gn/T/ipykernel_18060/1445966964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['testing_model'] = df1['testing_model_structure'] + '_' + df1['testing_model_type']+'_' + df1['testing_model_idx'].astype(str)
/var/folders/gt/llhq12_57w5fdzbm5g__rpj00000gn/T/ipykernel_18060/1445966964.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.drop(columns=['testing_model_structure', 'testing_model_type', 'testing_model_idx'], inplace=True)


,D_idx,acc,loss,testing_dataset,testing_model
8221,0,0.5111,2.609387,pgd_targeted_ll_ResNet20v1,ResNet20v1_S-B_0
8256,0,0.7688,2.357672,pgd_untargeted_ResNet20v1,ResNet20v1_S-B_0
8296,1,0.4822,2.742721,pgd_targeted_ll_ResNet20v1,ResNet20v1_S-B_1
8315,1,0.7871,2.405715,pgd_untargeted_ResNet20v1,ResNet20v1_S-B_1
8403,2,0.4914,2.671431,pgd_targeted_ll_ResNet20v1,ResNet20v1_S-B_2
...,...,...,...,...,...
11146,1,0.2025,6.402960,pgd_targeted_ll_ResNet56v2,ResNet56v2_B_1
11220,2,0.2060,6.107372,pgd_targeted_ll_ResNet56v2,ResNet56v2_B_2
11263,2,0.7305,6.821687,pgd_untargeted_ResNet56v2,ResNet56v2_B_2
11307,3,0.7499,6.307493,pgd_untargeted_ResNet56v2,ResNet56v2_B_3


In [121]:
df1.drop(columns=['D_idx'], inplace=True)


/var/folders/gt/llhq12_57w5fdzbm5g__rpj00000gn/T/ipykernel_18060/3801844372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.drop(columns=['D_idx'], inplace=True)


In [ ]:
pivot_df1 = df1.drop(columns=['loss']).pivot(index='testing_model', columns='testing_dataset', values = 'acc')


testing_dataset,cwl2_targeted_2nd_ResNet56v1,cwl2_targeted_ll_ResNet56v1,cwl2_untargeted_ResNet56v1,pgd_targeted_ll_ResNet20v1,pgd_targeted_ll_ResNet20v2,pgd_targeted_ll_ResNet56v1,pgd_targeted_ll_ResNet56v2,pgd_untargeted_ResNet20v1,pgd_untargeted_ResNet20v2,pgd_untargeted_ResNet56v1,pgd_untargeted_ResNet56v2
testing_model,,,,,,,,,,,
ResNet20v1_B_0,NaN,NaN,NaN,0.2070,NaN,NaN,NaN,0.7542,NaN,NaN,NaN
ResNet20v1_B_1,NaN,NaN,NaN,0.2143,NaN,NaN,NaN,0.7796,NaN,NaN,NaN
ResNet20v1_B_2,NaN,NaN,NaN,0.2072,NaN,NaN,NaN,0.7814,NaN,NaN,NaN
ResNet20v1_B_3,NaN,NaN,NaN,0.1816,NaN,NaN,NaN,0.7872,NaN,NaN,NaN
ResNet20v1_B_4,NaN,NaN,NaN,0.1803,NaN,NaN,NaN,0.7765,NaN,NaN,NaN
ResNet20v1_B_5,NaN,NaN,NaN,0.2032,NaN,NaN,NaN,0.7876,NaN,NaN,NaN
ResNet20v1_B_6,NaN,NaN,NaN,0.2020,NaN,NaN,NaN,0.7733,NaN,NaN,NaN
ResNet20v1_B_7,NaN,NaN,NaN,0.2143,NaN,NaN,NaN,0.7594,NaN,NaN,NaN
ResNet20v1_S-B_0,NaN,NaN,NaN,0.5111,NaN,NaN,NaN,0.7688,NaN,NaN,NaN


In [125]:
pivot_df1.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in pivot_df1.columns]


In [139]:
pivot_df1.transpose().to_csv('../results/single_models/avg_of_B_and_SB_self_attack.csv')

In [141]:
df = pd.read_csv('../results/single_models/avg_of_B_and_SB_self_attack.csv')

In [142]:
df

,Unnamed: 0,ResNet20v1_B,ResNet20v1_S-B,ResNet20v2_B,ResNet20v2_S-B,ResNet56v1_B,ResNet56v1_S-B,ResNet56v2_B,ResNet56v2_S-B
0,cwl2_targeted_2nd_ResNet56v1,NaN,NaN,NaN,NaN,0.089300,0.8671,NaN,NaN
1,cwl2_targeted_ll_ResNet56v1,NaN,NaN,NaN,NaN,0.828250,0.9026,NaN,NaN
2,cwl2_untargeted_ResNet56v1,NaN,NaN,NaN,NaN,0.088700,0.5625,NaN,NaN
3,pgd_targeted_ll_ResNet20v1,0.201238,0.486850,NaN,NaN,NaN,NaN,NaN,NaN
4,pgd_targeted_ll_ResNet20v2,NaN,NaN,0.199425,0.58800,NaN,NaN,NaN,NaN
5,pgd_targeted_ll_ResNet56v1,NaN,NaN,NaN,NaN,0.223875,0.5340,NaN,NaN
6,pgd_targeted_ll_ResNet56v2,NaN,NaN,NaN,NaN,NaN,NaN,0.207350,0.595200
7,pgd_untargeted_ResNet20v1,0.774900,0.783813,NaN,NaN,NaN,NaN,NaN,NaN
8,pgd_untargeted_ResNet20v2,NaN,NaN,0.744750,0.77685,NaN,NaN,NaN,NaN
9,pgd_untargeted_ResNet56v1,NaN,NaN,NaN,NaN,0.778067,0.7948,NaN,NaN


In [144]:
pivot_df_aggregated.transpose()

,B,S-B
cwl2_targeted_2nd_ResNet56v1,0.786051,0.894869
cwl2_targeted_ll_ResNet56v1,0.824821,0.906602
cwl2_untargeted_ResNet56v1,0.513648,0.577115
original_original_original,0.834513,0.909803
pgd_targeted_2nd_ResNet56v1,0.375219,0.561091
pgd_targeted_ll_ResNet20v1,0.510594,0.710166
pgd_targeted_ll_ResNet20v2,0.531371,0.725438
pgd_targeted_ll_ResNet56v1,0.506472,0.710215
pgd_targeted_ll_ResNet56v2,0.526864,0.723670
pgd_targeted_ll_vgg11,0.654498,0.795841
